In [1]:
import sys
import os
import pyxodr

In [2]:
# # Install CARLA Python API (if not already installed)
# # Note: This assumes you have the CARLA .egg file or wheel. Adjust the path as needed.


# # Example: Add CARLA egg to sys.path (update the path to your CARLA install)
# carla_egg = None
# for root, dirs, files in os.walk('C:/carla/PythonAPI/carla/dist'):
#     for file in files:
#         if file.startswith('carla-') and file.endswith('.egg'):
#             carla_egg = os.path.join(root, file)
#             break
#     if carla_egg:
#         break
# if carla_egg and carla_egg not in sys.path:
#     sys.path.append(carla_egg)

# try:
#     import carla
# except ImportError:
#     print('CARLA egg not found or CARLA not installed. Please check your CARLA installation path.')

# # Connect to CARLA server and load Town01
# try:
#     client = carla.Client('localhost', 2000)
#     client.set_timeout(10.0)
#     world = client.load_world('Town01')
#     print('Loaded map:', world.get_map().name)
# except Exception as e:
#     print('Error connecting to CARLA or loading map:', e)

In [3]:
carla_map_dir = 'C:\Carla-0.10.0\CarlaUnreal\Content\Carla\Maps\OpenDrive'

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\91747\AppData\Local\Temp\ipykernel_11764\2772292095.py:1: SyntaxWarning: invalid escape sequence '\C'
  carla_map_dir = 'C:\Carla-0.10.0\CarlaUnreal\Content\Carla\Maps\OpenDrive'


In [4]:
import os
from lxml import etree
import pyxodr
from pyxodr.road_objects.road import Road
from pyxodr.road_objects.lane import Lane,ConnectionPosition,LaneOrientation, TrafficOrientation
from pyxodr.road_objects.junction import Junction
from pyxodr.road_objects.lane_section import LaneSection
from pyxodr.road_objects.network import RoadNetwork
from shapely.geometry import Polygon
from enum import IntEnum


In [5]:
carla_map_dir = 'C:\\Carla-0.10.0\\CarlaUnreal\\Content\\Carla\\Maps\\OpenDrive'
odr_file = os.path.join(carla_map_dir, 'Town01.xodr')

road_network = RoadNetwork(xodr_file_path=odr_file)

In [5]:
roads = road_network.get_roads()
print(f"Number of roads in the network: {len(roads)}")
print(f"Type: {type(roads[0])}\nRoads: {roads}")


Number of roads in the network: 122
Type: <class 'pyxodr.road_objects.road.Road'>
Roads: [Road_0, Road_1, Road_2, Road_3, Road_4, Road_5, Road_6, Road_7, Road_8, Road_9, Road_10, Road_11, Road_12, Road_13, Road_14, Road_15, Road_16, Road_17, Road_18, Road_19, Road_20, Road_21, Road_22, Road_23, Road_24, Road_25, Road_27, Road_28, Road_33, Road_40, Road_41, Road_46, Road_48, Road_52, Road_61, Road_62, Road_67, Road_70, Road_75, Road_77, Road_83, Road_85, Road_93, Road_94, Road_98, Road_102, Road_103, Road_104, Road_108, Road_109, Road_117, Road_118, Road_123, Road_128, Road_131, Road_133, Road_139, Road_141, Road_144, Road_145, Road_150, Road_157, Road_158, Road_163, Road_165, Road_169, Road_178, Road_179, Road_183, Road_184, Road_185, Road_188, Road_189, Road_193, Road_195, Road_197, Road_201, Road_203, Road_208, Road_209, Road_218, Road_219, Road_223, Road_229, Road_232, Road_243, Road_244, Road_247, Road_248, Road_253, Road_256, Road_257, Road_258, Road_271, Road_272, Road_273, Road_

Road Edges Extraction

In [7]:
# for road_obj in roads:
#     print(f"Road ID: {road_obj.id}")
#     lane_sections = road_obj.lane_sections
#     print(f"Lane Sections: {lane_sections}")
#     road_obj.lane_borders = {
#         "left": [left_lane.boundary_line for left_lane in road_obj.lane_sections[0].left_lanes],
#         "right": [right_lane.boundary_line for right_lane in road_obj.lane_sections[0].right_lanes]
#     }
#     road_edge = road_obj.boundary
#     print(f"Road edges in {road_obj.id}: {road_edge}")
#     break

In [7]:
class MapType(IntEnum):
    LANE_UNDEFINED = 0
    LANE_FREEWAY = 1
    LANE_SURFACE_STREET = 2
    LANE_BIKE_LANE = 3
    # Original definition skips 4
    ROAD_LINE_UNKNOWN = 5
    ROAD_LINE_BROKEN_SINGLE_WHITE = 6
    ROAD_LINE_SOLID_SINGLE_WHITE = 7
    ROAD_LINE_SOLID_DOUBLE_WHITE = 8
    ROAD_LINE_BROKEN_SINGLE_YELLOW = 9
    ROAD_LINE_BROKEN_DOUBLE_YELLOW = 10
    ROAD_LINE_SOLID_SINGLE_YELLOW = 11
    ROAD_LINE_SOLID_DOUBLE_YELLOW = 12
    ROAD_LINE_PASSING_DOUBLE_YELLOW = 13
    ROAD_EDGE_UNKNOWN = 14
    ROAD_EDGE_BOUNDARY = 15
    ROAD_EDGE_MEDIAN = 16
    STOP_SIGN = 17
    CROSSWALK = 18
    SPEED_BUMP = 19
    DRIVEWAY = 20  # New womd datatype in v1.2.0: Driveway entrances
    UNKNOWN = -1
    NUM_TYPES = 21


In [17]:
import json

# Initialize the JSON structure
xodr_json = {
    "roads": []
}

# Save to file (empty roads list for now)
with open("town01.json", "w") as f:
    json.dump(xodr_json, f, indent=2)

In [19]:
def save_lane_section_to_json(xodr_json, id, road_edges, road_lines, lanes, sidewalks = []):
    for road_edge in road_edges:
        # edge_polygon = Polygon(road_edge)
        edge_data = {
            "id": id,
            "map_element_id": int(MapType.ROAD_EDGE_BOUNDARY),
            "type": "road_edge",
            "geometry": [{"x": float(pt[0]), "y": float(pt[1]), "z": 0.0} for pt in road_edge]
        }
        xodr_json["roads"].append(edge_data)
        id += 1
    for road_line in road_lines:
        line_data = {
            "id": id,
            "map_element_id": int(MapType.ROAD_LINE_BROKEN_SINGLE_WHITE),
            "type": "road_line",
            "geometry": [{"x": float(pt[0]), "y": float(pt[1]), "z": 0.0} for pt in road_line]
        }
        xodr_json["roads"].append(line_data)
        id += 1
    for lane in lanes:
        lane_data = {
            "id": id,
            "map_element_id": int(MapType.LANE_SURFACE_STREET),
            "type": "lane",
            "geometry": [{"x": float(pt[0]), "y": float(pt[1]), "z": 0.0} for pt in lane]
        }
        xodr_json["roads"].append(lane_data)
        id += 1
    # for sidewalk in sidewalks:
    #     sidewalk_data = {
    #         "id": id,
    #         "map_element_id": int(MapType.LANE_BIKE_LANE),
    #         "type": "sidewalk",
    #         "geometry": [{"x": float(pt[0]), "y": float(pt[1]), "z": 0.0} for pt in sidewalk]
    #     }
    #     xodr_json["roads"].append(sidewalk_data)
    #     id += 1
    return id

Road Edges, Road Lines, Lanes and Sidewalks extraction

In [20]:
# TODO: Go only till last "driving" lane("parking" NTD)
# "median" lane means a road edge(add after all of them appear)
# Add "sidewalk" lane as well

id = 0
for road_obj in roads:
    # print(f"Road ID: {road_obj.id}")
    lane_sections = road_obj.lane_sections
    # print(f"Lane Sections: {lane_sections}")
    for lane_section in lane_sections:
        # print(f"Lane Section ID: {lane_section.lane_section_ordinal}")
        # print(f"Number of Left Lanes: {len(lane_section.left_lanes)}")
        # print(f"Number of Right Lanes: {len(lane_section.right_lanes)}")
        road_edges = []     # First add center lane as road_reference line
        road_lines = []
        lanes = []
        # sidwalks = []
        
        # Left Lanes
        add_lane_data = False
        add_edge_data = False
        previous_lane = None
        for i, left_lane in enumerate(lane_section.left_lanes):
            if left_lane.type == 'driving' or left_lane.type == 'parking':
                if add_lane_data:
                    road_lines.append(previous_lane.boundary_line)
                    lanes.append(previous_lane.centre_line)
                # Add outer edge as road edge
                elif add_edge_data:
                    road_edges.append(previous_lane.boundary_line)
                add_lane_data = True
                add_edge_data = False
            else:
                # Add inner lane as road edge
                if add_lane_data and i != 0:
                    road_edges.append(previous_lane.boundary_line)
                add_edge_data = True
                add_lane_data = False
            previous_lane = left_lane

        if add_lane_data:
            road_edges.append(previous_lane.boundary_line)
        # elif add_edge_data:
        #     if previous_lane.type == 'sidewalk':
                # sidwalks.append(previous_lane.boundary_line)
        
        # print("LEFT STATS")
        # print(f"Number of Road edges: {len(road_edges)}")
        # print(f"Road lines: {len(road_lines)}")
        # print(f"Lanes: {len(lanes)}")
        # print(f"Sidewalks: {len(sidwalks)}")

        # Right Lanes
        add_lane_data = False
        add_edge_data = False
        previous_lane = None
        for i, right_lane in enumerate(lane_section.right_lanes):
            if right_lane.type == 'driving' or right_lane.type == 'parking':
                if add_lane_data:
                    road_lines.append(previous_lane.boundary_line)
                    lanes.append(previous_lane.centre_line)
                # Add outer edge as road edge
                elif add_edge_data:
                    road_edges.append(previous_lane.boundary_line)
                add_lane_data = True
                add_edge_data = False
            else:
                # Add inner lane as road edge
                if add_lane_data and i != 0:
                    road_edges.append(previous_lane.boundary_line)
                add_edge_data = True
                add_lane_data = False
            previous_lane = right_lane

        if add_lane_data:
            road_edges.append(previous_lane.boundary_line)
        # elif add_edge_data:
        #     if previous_lane.type == 'sidewalk':
        #         sidwalks.append(previous_lane.boundary_line)

        # print(f"Number of Road edges in {road_obj.id}: {len(road_edges)}")
        # print(f"Road lines in {road_obj.id}: {len(road_lines)}")
        # print(f"Lanes in {road_obj.id}: {len(lanes)}")
        # print(f"Sidewalks in {road_obj.id}: {len(sidwalks)}")
        if len(lanes) > 0:
            print(f"Number of lanes in {road_obj.id}: {len(lanes)}")
            print(lanes[0].shape)
        
        id = save_lane_section_to_json(xodr_json, id, road_edges, road_lines, lanes)
    #     break
    # break

In [21]:
# Save to file
with open("town01.json", "w") as f:
    json.dump(xodr_json, f, indent=2)

In [ ]:
print(len(road_edges))
print(road_edges[0].shape)
# print(road_lines[0].shape)
# print(lanes[0].shape)
# print(sidwalks[0].shape)
print(road_edges)
# print(road_lines)
# print(lanes)
# print(sidwalks)

1
(222, 2)
(222, 2)
[array([[144.99048304, -61.49334925],
       [145.09084503, -61.49333702],
       [145.19120702, -61.49332479],
       [145.29156901, -61.49331256],
       [145.391931  , -61.49330033],
       [145.49229299, -61.4932881 ],
       [145.59265498, -61.49327587],
       [145.69301697, -61.49326364],
       [145.79337896, -61.49325141],
       [145.89374095, -61.49323918],
       [145.99410294, -61.49322695],
       [146.09446493, -61.49321472],
       [146.19482692, -61.49320249],
       [146.29518891, -61.49319026],
       [146.3955509 , -61.49317803],
       [146.49591289, -61.4931658 ],
       [146.59627488, -61.49315358],
       [146.69663687, -61.49314135],
       [146.79699886, -61.49312912],
       [146.89736085, -61.49311689],
       [146.99772284, -61.49310466],
       [147.09808483, -61.49309243],
       [147.19844682, -61.4930802 ],
       [147.29880881, -61.49306797],
       [147.3991708 , -61.49305574],
       [147.49953279, -61.49304351],
       [147.59989

In [7]:
import carla
import numpy as np

# Connect to CARLA
client = carla.Client('localhost', 2000)
# If you want to ensure you're using Town01 (to match your OpenDRIVE file), uncomment the next line:
world = client.load_world('Town01')
# world = client.get_world()
current_map = world.get_map()
print("Currently loaded map:", current_map.name)

# Spawn a vehicle
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
spawn_points = current_map.get_spawn_points()
vehicle = world.spawn_actor(vehicle_bp, spawn_points[0])

# Enable autopilot
vehicle.set_autopilot(True)

# Log trajectory
trajectory = []
for _ in range(1000):  # Log for 1000 steps
    world.tick()
    transform = vehicle.get_transform()
    velocity = vehicle.get_velocity()
    
    trajectory.append({
        "position": {"x": transform.location.x, "y": transform.location.y, "z": transform.location.z},
        "heading": transform.rotation.yaw,
        "velocity": {"x": velocity.x, "y": velocity.y}
    })

# Save trajectory
import json
with open("expert_trajectory.json", "w") as f:
    json.dump(trajectory, f, indent=2)

RuntimeError: No connection could be made because the target machine actively refused it.

In [10]:
# try:
#     if os.path.exists(odr_file):
#         tree = etree.parse(odr_file)
#         root = tree.getroot()
#         # Find all <road> elements
#         road_elements = root.findall('road')
#         # Example: create a pyxodr Road object for the first road
#         if road_elements:
#             print(type(road_elements))
#             print(type(road_elements[0]))
#             road_obj = Road(road_elements[7])
#             print(f'Successfully created pyxodr Road object: {road_obj}')
#         else:
#             print('No <road> elements found in the file.')
#     else:
#         print('OpenDRIVE file not found:', odr_file)
# except Exception as e:
#     print('Error reading OpenDRIVE file or creating Road object:', e)

In [11]:
print(road_obj.lane_sections)

[Section_0/Road_355]


In [12]:
# For a given Road object (road_obj)
import numpy as np

def get_lane_centerline(lane, lane_section_ref_line, lane_widths):
    """
    Compute the centerline of a lane.
    lane: pyxodr Lane object
    lane_section_ref_line: np.ndarray, reference line for the lane section
    lane_widths: np.ndarray, width of the lane at each point
    """
    # t_s = offset from reference line to lane center
    # For right lanes, offset is negative; for left, positive
    # Lane center = start offset + 0.5 * lane width
    # Lane offset is cumulative sum of all previous lane widths (from center out)
    # pyxodr Lane has .side ('left' or 'right') and .widths (list of width polynomials)
    # We'll use lane.centerline if available, else compute manually
    if hasattr(lane, 'centerline'):
        return lane.centerline
    else:
        # Fallback: approximate centerline
        # This is a simplification; for full accuracy, use pyxodr's built-in methods
        return lane_section_ref_line + 0.5 * lane_widths[:, None]


In [13]:
road_edges = road_obj.boundary

# road_obj is your Road object
lane_borders = road_obj.lane_borders  # dict with keys "left" and "right"
left_lane_markings = lane_borders["left"]   # List of np.ndarray, each is a border line
right_lane_markings = lane_borders["right"] # List of np.ndarray, each is a border line

internal_left_markings = road_obj.lane_borders["left"][:-1]
internal_right_markings = road_obj.lane_borders["right"][:-1]
road_lines = internal_left_markings + internal_right_markings
lane_centerlines = []

print(f"Road edges in {road_obj.id}: {road_edges}")
print(f"Road Lines in {road_obj.id}: {road_lines}")


AttributeError: 'Road' object has no attribute 'lane_borders'

In [ ]:


for lane_section in road_obj.lane_sections:
    # Get the reference line for this lane section
    ref_line = lane_section.lane_section_reference_line  # shape (N, 2)
    # Get all lanes in this section (dict: id -> Lane)
    lanes = lane_section.lanes
    # Get Road_Edges
    road_edges = lane_section.boundary
    print(f"Road edges in {lane_section.road_id} section {lane_section.lane_section_ordinal}: {road_edges}")
    # Sort lane ids: center (0), then left (1,2,...) and right (-1,-2,...)
    lane_ids = sorted(lanes.keys(), key=lambda x: (abs(int(x)), -int(x)))
    # For each lane, get its border and centerline
    for lane_id in lane_ids:
        lane = lanes[lane_id]
        # Lane border (left/right edge)
        if hasattr(lane, 'border'):
            border = lane.border  # shape (N, 2)
        else:
            border = None
        # Lane width at each point
        if hasattr(lane, 'widths'):
            # widths is a list of polynomials, but pyxodr usually provides .width_line
            lane_widths = lane.width_line  # shape (N,)
        else:
            lane_widths = None
        # Lane centerline
        centerline = get_lane_centerline(lane, ref_line, lane_widths) if lane_widths is not None else None
        # Save centerline
        if centerline is not None:
            lane_centerlines.append((lane_id, centerline))
        # Save border as road_edge if this is the outermost drivable lane
        if border is not None and lane.type == 'driving':
            # Check if this is the outermost left/right drivable lane
            if (lane_id == max([int(l) for l in lanes if lanes[l].type == 'driving'])) or \
               (lane_id == min([int(l) for l in lanes if lanes[l].type == 'driving'])):
                road_edges.append((lane_id, border))
        # Save border as road_line (lane marking) if not the outermost lane
        if border is not None and lane.type == 'driving':
            if not ((lane_id == max([int(l) for l in lanes if lanes[l].type == 'driving'])) or \
                    (lane_id == min([int(l) for l in lanes if lanes[l].type == 'driving']))):
                road_lines.append((lane_id, border))

# Example: print the number of each feature
print(f"Extracted {len(road_edges)} road edges, {len(road_lines)} road lines, {len(lane_centerlines)} lane centerlines.")

# Each entry is (lane_id, np.ndarray of shape (N,2))
# You can plot or save these as needed

Road edges in 7 section 0: POLYGON ((347.7299857674992 -320.3099853515625, 347.8301235107584 -320.3099853515625, 347.93026125401764 -320.3099853515625, 348.0303989972769 -320.3099853515625, 348.1305367405361 -320.3099853515625, 348.2306744837953 -320.3099853515625, 348.33081222705454 -320.3099853515625, 348.43094997031375 -320.3099853515625, 348.53108771357296 -320.3099853515625, 348.6312254568322 -320.3099853515625, 348.73136320009144 -320.3099853515625, 348.83150094335065 -320.3099853515625, 348.93163868660986 -320.3099853515625, 349.03177642986907 -320.3099853515625, 349.1319141731283 -320.3099853515625, 349.23205191638755 -320.3099853515625, 349.33218965964676 -320.3099853515625, 349.43232740290597 -320.3099853515625, 349.5324651461652 -320.3099853515625, 349.6326028894244 -320.3099853515625, 349.73274063268366 -320.3099853515625, 349.83287837594287 -320.3099853515625, 349.9330161192021 -320.3099853515625, 350.0331538624613 -320.3099853515625, 350.1332916057205 -320.3099853515625, 

AttributeError: 'list' object has no attribute 'keys'